In [27]:
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten
from keras.utils import np_utils
import scipy
from keras.layers import Dense
from snorkel.labeling import labeling_function
from snorkel.labeling import LFAnalysis
import pandas as pd
import glob
import os
import subprocess
from collections import OrderedDict
import torch
import torch.nn as nn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from snorkel.classification.data import DictDataset, DictDataLoader
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(5)
from snorkel.labeling import model

In [28]:
def load_spam_dataset_2(load_train_labels: bool = False, split_dev: bool = True):
    dfs = []
    df = pd.read_csv('.\\data\\processed_data_labled.csv')
    #keys = ['x.BPFI','x.BPFO','x.FTF','x.BSF','x.hi_spectr.sp_entropy']
    #df = [df.get(key) for key in keys]
    # Lowercase column names
    df.columns = map(str.lower, df.columns)
    # Remove comment_id field
    df = df.drop("id", axis=1)
    # Add field indicating source video
    # Shuffle order
    
    df = df.rename(columns={"state": "label",
                            "y.ar6":"y_ar6",
                            "x.hi_spectr.sp_entropy":"x_hi_spectr_sp_entropy",
                            "x.p2p":"x_p2p",
                            "x.freq.7":"x_freq_7",
                            "y.ar4":"y_ar4",
                            "x.ar5":"x_ar5",
                            "y.ar8":"y_ar8",
                            "x.hi_spectr.rmsf":"x_hi_spectr_rmsf",
                            "y.hi_spectr.sp_sd":"y_hi_spectr_sp_sd",
                            "x.hi_spectr.sp_mean":"x_hi_spectr_sp_mean",
                            "y.ar1":"y_ar1",
                            "x.hi_spectr.sp_kurtosis":"x_hi_spectr_sp_kurtosis",
                            "y.low_spectr.vf":"y_low_spectr_vf",
                            "x.rms":"x_rms"})
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    dfs.append(df)
    df_train = pd.concat(dfs)
    if split_dev:
        df_dev = df_train.sample(8624, random_state=123)
#     print("dfs : ", dfs)
    df_valid_test = dfs[:11]
    print("df_valid_test : ",df_valid_test)
    df_valid, df_test = train_test_split(
        df, test_size=1000, random_state=123)

    if split_dev:
        return df_train, df_dev, df_valid, df_test
    else:
        return df_train, df_valid, df_test
    

df_train, df_dev, df_valid, df_test = load_spam_dataset_2()
Y_dev = df_dev.label.values
Y_valid = df_valid.label.values
Y_test = df_test.label.values

#df_train = pd.concat(dataset)
len(df_train)

df_valid_test :  [        x_mean    y_mean      x_sd      y_sd  x_skewness  y_skewness  \
0    -0.116163 -0.115837  0.096711  0.086653    0.004068    0.061271   
1    -0.113949 -0.113511  0.073493  0.080967    0.092004    0.072770   
2    -0.095569 -0.094902  0.090742  0.077866   -0.024963    0.142320   
3    -0.114062 -0.114666  0.114355  0.116910    0.075810    0.020114   
4    -0.117847 -0.117721  0.099753  0.081485   -0.025136   -0.057403   
...        ...       ...       ...       ...         ...         ...   
8619 -0.114192 -0.113695  0.070723  0.076371    0.003236    0.095189   
8620 -0.113973 -0.113317  0.084753  0.083828    0.022790    0.058452   
8621 -0.112926 -0.113750  0.113858  0.115721   -0.009496   -0.007443   
8622 -0.117157 -0.117494  0.098448  0.088732    0.009244    0.040438   
8623 -0.117149 -0.117244  0.092653  0.076886   -0.092272   -0.057444   

      x_kurtosis  y_kurtosis  x_entropy  y_entropy  ...  y.hi_spectr.fc  \
0       1.107000    0.831559   5.018915   

8624

In [29]:
from snorkel.labeling import PandasLFApplier
@labeling_function()
def snorkel_label_function_1(x):
    if x.y_ar6 >= 0.0119 and x.y_ar6 <= 0.0795:
        return 1
    return 0

@labeling_function()
def snorkel_label_function_2(x):
    if x.y_ar6 >= -0.0376 and x.y_ar6 < 0.0119:
        return 1
    return 0

@labeling_function()
def snorkel_label_function_3(x):
    if x.y_ar6 >= -0.0376 and x.y_ar6 < 0.0119 and x.x_hi_spectr_sp_entropy > 5.5690:
        return 1
    return 0


@labeling_function()
def snorkel_label_function_4(x):
    if x.y_ar6 >= 0.011944 and x.y_ar6 < 0.079553 and x.x_p2p >= 1.0400 and x.x_p2p < 1.2410:
        return 1
    return 0

@labeling_function()
def snorkel_label_function_5(x):
    if x.y_ar6 >= 0.011944 and x.y_ar6 < 0.079553 and x.x_p2p >= 1.0400 and x.x_p2p < 1.2410 and x.x_freq_7 <=2480.2480 :
        return 1
    return 0

@labeling_function()
def snorkel_label_function_6(x):
    if x.y_ar6 >= 0.011944 and x.y_ar6 < 0.079553 and x.x_p2p > 1.2410 and  x.y_ar4 <= 0.0682:
        return 1
    return 0

@labeling_function()
def snorkel_label_function_7(x):
    if x.y_ar6 >= 0.07955 and x.y_ar6 < 0.12033 :
        return 1
    return 0


@labeling_function()
def snorkel_label_function_8(x):
    if x.y_ar6 >= 0.07955 and x.y_ar6 < 0.12033  and x.x_ar5 <=-0.05911:
        return 1
    return 0


lfs = [
       snorkel_label_function_1
       ,snorkel_label_function_2
       ,snorkel_label_function_3
       ,snorkel_label_function_4
       ,snorkel_label_function_5
       ,snorkel_label_function_6
       ,snorkel_label_function_7
       ,snorkel_label_function_8
      ]
# lfs = [check_y_ar4,check_x_ar6,check_y_ar8,check_x_hi_spectr_rmsf,check_x_hi_spectr_sp_sd,check_x_hi_spectr_sp_mean,check_y_ar1,check_x_hi_spectr_sp_kurtosis,
#       check_x_rms]
# lfs = [check_y_ar4,check_x_hi_spectr_sp_mean,check_x_rms]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)
L_valid = applier.apply(df=df_valid)
LFAnalysis(L=L_train, lfs=lfs).lf_summary()
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

100%|███████████████████████████████████████████████████████████████████████████| 7624/7624 [00:00<00:00, 12664.48it/s]


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
snorkel_label_function_1,0,"[0, 1]",1.0,1.0,0.299861,8241,383,0.955589
snorkel_label_function_2,1,"[0, 1]",1.0,1.0,0.299861,7280,1344,0.844156
snorkel_label_function_3,2,"[0, 1]",1.0,1.0,0.299861,8004,620,0.928108
snorkel_label_function_4,3,"[0, 1]",1.0,1.0,0.299861,7780,844,0.902134
snorkel_label_function_5,4,"[0, 1]",1.0,1.0,0.299861,7843,781,0.909439
snorkel_label_function_6,5,"[0, 1]",1.0,1.0,0.299861,8418,206,0.976113
snorkel_label_function_7,6,"[0, 1]",1.0,1.0,0.299861,7008,1616,0.812616
snorkel_label_function_8,7,"[0, 1]",1.0,1.0,0.299861,7802,822,0.904685


In [30]:
label_model = model.LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, lr=0.001, log_freq=100, seed=123)
label_model_acc = label_model.score(L=L_valid, Y=Y_valid, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")
label_model.get_weights()
label_model.save(".\\data\\label_model\\snorkel_model")

Label Model Accuracy:     91.1%


C:\Users\iktus\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [31]:
L_test = applier.apply(df=df_train)
label_model.predict(L_test)
L_train = applier.apply(df=df_train)
result = label_model.predict(L_train)
df_train.to_csv('.\\data\\maked_data.csv',sep=',',na_rep='NaN')

100%|███████████████████████████████████████████████████████████████████████████| 8624/8624 [00:00<00:00, 12626.62it/s]
